In [30]:
import whisper
import os
import requests

try:
    response = requests.get("https://huggingface.co", timeout=20, verify=False)  # Disable SSL verification
except requests.exceptions.ReadTimeout:
    print("La solicitud ha tardado demasiado en responder.")
except requests.exceptions.SSLError as e:
    print(f"SSL error: {e}")

print(os.getcwd())
model = whisper.load_model("base")
file_path = "c:\\Users\\PGCARL6\\Desktop\\Mis cosas\\TFG\\System-for-classifying-podcasts-into-themes-using-large-language-models\\data\\audio.wav"
print(file_path)
if os.path.exists(file_path):
    print("El archivo existe.")
else:
    print(f"No se encuentra el archivo en: {file_path}")
try:
    with open(file_path, "rb") as f:
        print("El archivo se puede leer.")
except Exception as e:
    print(f"No se pudo leer el archivo: {e}")

try:
    result = model.transcribe(file_path, fp16=False)
    transcript = result["text"]
    print(transcript)
except FileNotFoundError:
    print("El archivo especificado no se encuentra.")
except requests.exceptions.ReadTimeout:
    print("La solicitud ha tardado demasiado en responder.")

c:\Users\PGCARL6\Desktop\Mis cosas\TFG\System-for-classifying-podcasts-into-themes-using-large-language-models\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


c:\Users\PGCARL6\Desktop\Mis cosas\TFG\System-for-classifying-podcasts-into-themes-using-large-language-models
c:\Users\PGCARL6\Desktop\Mis cosas\TFG\System-for-classifying-podcasts-into-themes-using-large-language-models\data\audio.wav
El archivo existe.
El archivo se puede leer.
El archivo especificado no se encuentra.


In [22]:
import os

# Imprime el directorio de trabajo actual
print("Directorio de trabajo actual:", os.getcwd())

# Verifica si el archivo existe
file_path = 'data/audio.ogg'
if os.path.exists(file_path):
    print("El archivo existe.")
else:
    print("El archivo no existe.")

Directorio de trabajo actual: c:\Users\PGCARL6\Desktop\Mis cosas\TFG\System-for-classifying-podcasts-into-themes-using-large-language-models
El archivo existe.


In [ ]:
import pandas as pd

# Ejemplo de datos
data = {
    "podcast_id": [1, 2, 3],
    "transcript": ["Transcripción del podcast 1", "Transcripción del podcast 2", "Transcripción del podcast 3"],
    "label": ["Tecnología", "Salud", "Educación"]
}

df = pd.DataFrame(data)
df.to_csv("podcast_transcripts.csv", index=False)
# Cargar y verificar el CSV
df = pd.read_csv("podcast_transcripts.csv")
print(df.head())

##  Preparación de Datos

In [2]:
import pandas as pd
from datasets import Dataset

# Cargar datos
data = pd.read_csv('podcast_transcripts.csv')
dataset = Dataset.from_pandas(data)

c:\Users\PGCARL6\Desktop\Mis cosas\TFG\System-for-classifying-podcasts-into-themes-using-large-language-models\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'podcast_transcripts.csv'

## Tokenización

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['transcript'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 45305f42-baf6-4b80-96bf-ed1207bba69d)')

## Cargar el Modelo Preentrenado

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['label'].unique()))

# Dividir el conjunto de datos en entrenamiento y prueba
train_test = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test['train']
test_dataset = train_test['test']

## Entrenamiento del Modelo

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

## Evaluación del Modelo

In [ ]:
results = trainer.evaluate()
print(results)